### Предобработка текста

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("https://storage.yandexcloud.net/auth-def-2024/datasets/meta_table_with_texts.csv")
df = df[['author', 'text']]
df.head(5)

,author,text
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...


Загружаем лемматизированные тексты из заранее подготовленного файла, сформированного после экспериментов.

In [3]:
df_lemm = pd.read_csv('lemm_texts.csv')
df_lemm.head()

,author,lemm_text
0,Пушкин Александр Сергеевич,литературный альбомъ сразить рыцарь послднимъ ...
1,Карамзин Николай Михайлович,достоинство древний новый перевод немецкий нек...
2,Гоголь Николай Васильевич,полный собрание сочинение письмо так переписка...
3,Мамин-Сибиряк Дмитрий Наркисович,собрана сочиненйтомъ восьмой издан марксъ петр...
4,Мамин-Сибиряк Дмитрий Наркисович,нимфа щегольский волжский пароход вулкан дать ...


In [4]:
df['lemm_text'] = df_lemm['lemm_text']
df.head()

,author,text,lemm_text
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ сразить рыцарь послднимъ ...
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,достоинство древний новый перевод немецкий нек...
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,полный собрание сочинение письмо так переписка...
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,собрана сочиненйтомъ восьмой издан марксъ петр...
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,нимфа щегольский волжский пароход вулкан дать ...


In [5]:
# Удаление невалидной записи с английским текстом
df = df.drop(index=1918)
df = df.reset_index(drop=True)
len(df)

2564

In [6]:
# Токенизация текста
def words_list(x):
  # Подсчёт слов в тексте
  words = re.findall(r'\b\w+\b', x)
  return [w.lower() for w in words]

In [7]:
df['words'] = df['lemm_text'].apply(words_list)
df.head(5)

,author,text,lemm_text,words
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ сразить рыцарь послднимъ ...,"[литературный, альбомъ, сразить, рыцарь, послд..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,достоинство древний новый перевод немецкий нек...,"[достоинство, древний, новый, перевод, немецки..."
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,полный собрание сочинение письмо так переписка...,"[полный, собрание, сочинение, письмо, так, пер..."
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,собрана сочиненйтомъ восьмой издан марксъ петр...,"[собрана, сочиненйтомъ, восьмой, издан, марксъ..."
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,нимфа щегольский волжский пароход вулкан дать ...,"[нимфа, щегольский, волжский, пароход, вулкан,..."


Загрузим эвристики, которые были рассчитаны и отобраны на этапе проведения экспериментов и замера метрик качества моделей.

In [8]:
# Загрузка фрейма с эвристиками
df_heuristics = pd.read_csv('heuristics.csv')
df_heuristics.head(5)

,num_words,avg_sentence_length,avg_word_length
0,347,17.350000,5.002882
1,988,17.642857,5.452429
2,177463,13.357143,5.226318
3,8774,12.410184,5.284477
4,4221,12.234783,5.027008


### Формирование выборок

На этапе проведения экспериментов было принято решение использовать комбинированный подход в выборе признаков.

In [10]:
X_heuristics = df_heuristics.copy()
X_words = df['words'].copy()
y = df['author'].copy()

In [11]:
from sklearn.model_selection import train_test_split

# Обучающая и тренировочная выборки
X_heur_train, X_heur_test, X_words_train, X_words_test, y_train, y_test = train_test_split(X_heuristics, X_words, y, test_size=0.2, random_state=42)

### Формирование эмбеддингов

In [14]:
import numpy as np

# Функция для представления текста как среднего из векторов слов
def vectorize_text(text, model):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)  # Если нет слов из текста, возвращаем нулевой вектор
    return np.mean(vectors, axis=0)

Будем рассматривать $\text{Word2Vec}$, $\text{FastText}$ и $\text{GloVe}$, а также использовать $\text{Skip Gram}$.

#### Word2Vec

In [12]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=X_words_train, vector_size=300, window=5, min_count=1, workers=4)

In [15]:
X_train_vect_w2v = np.array([vectorize_text(text, w2v_model) for text in X_words_train])
X_test_vect_w2v = np.array([vectorize_text(text, w2v_model) for text in X_words_test])

In [16]:
X_train_combined_w2v = np.hstack([X_heur_train, X_train_vect_w2v])
X_test_combined_w2v = np.hstack([X_heur_test, X_test_vect_w2v])

Теперь с использованием $\text{Skip Gram}$:

In [13]:
w2v_model_sg = Word2Vec(sentences=X_words_train, vector_size=300, window=5, sg=1, min_count=1, workers=4)

In [17]:
X_train_vect_w2v_sg = np.array([vectorize_text(text, w2v_model_sg) for text in X_words_train])
X_test_vect_w2v_sg = np.array([vectorize_text(text, w2v_model_sg) for text in X_words_test])

In [18]:
X_train_combined_w2v_sg = np.hstack([X_heur_train, X_train_vect_w2v_sg])
X_test_combined_w2v_sg = np.hstack([X_heur_test, X_test_vect_w2v_sg])

#### GloVe

Для $\text{GloVe}$ будем использовать стандартные [предобученные эмбеддинги](http://vectors.nlpl.eu/repository/#) (Национальный корпус русского языка + русская Википедия, `vector_size=300, window=5`).

In [37]:
def load_glove_model(glove_path):
    glove_model = {}
    with open(glove_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)
            glove_model[word] = vector
    return glove_model

glove_path = "/content/model.txt"
glove_model = load_glove_model(glove_path)

In [38]:
# Функция для представления текста как среднего из векторов слов
def sentence_to_glove_embedding(sentence, glove_model, embedding_dim=100):
    vectors = [glove_model[word] for word in sentence if word in glove_model]
    if len(vectors) == 0:
        return np.zeros(embedding_dim)  # Если нет слов из текста, возвращаем нулевой вектор
    return np.mean(vectors, axis=0)

In [39]:
X_train_vect_glove = np.array([sentence_to_glove_embedding(sentence, glove_model) for sentence in X_words_train])
X_test_vect_glove = np.array([sentence_to_glove_embedding(sentence, glove_model) for sentence in X_words_test])

In [40]:
X_train_combined_glove = np.hstack([X_heur_train, X_train_vect_glove])
X_test_combined_glove = np.hstack([X_heur_test, X_test_vect_glove])

### FastText

In [ ]:
!pip install gensim

In [27]:
from gensim.models import FastText

model_fasttext = FastText(sentences=X_words_train, vector_size=300, window=5, min_count=1, workers=4, epochs=10)

In [28]:
X_train_vect_fasttext = np.array([vectorize_text(text, model_fasttext) for text in X_words_train])
X_test_vect_fasttext = np.array([vectorize_text(text, model_fasttext) for text in X_words_test])

In [29]:
X_train_combined_fasttext = np.hstack([X_heur_train, X_train_vect_fasttext])
X_test_combined_fasttext = np.hstack([X_heur_test, X_test_vect_fasttext])

Теперь с использованием $\text{Skip Gram}$:

In [32]:
model_fasttext_sg = FastText(sentences=X_words_train, vector_size=300, sg=1, window=5, min_count=1, workers=4, epochs=10)

In [33]:
X_train_vect_fasttext_sg = np.array([vectorize_text(text, model_fasttext_sg) for text in X_words_train])
X_test_vect_fasttext_sg = np.array([vectorize_text(text, model_fasttext_sg) for text in X_words_test])

In [34]:
X_train_combined_fasttext_sg = np.hstack([X_heur_train, X_train_vect_fasttext_sg])
X_test_combined_fasttext_sg = np.hstack([X_heur_test, X_test_vect_fasttext_sg])

### Сравнение пайплайнов на разных эмбеддингах

На этапе проведения экспериментов было выявлено, что лучшие метрики качества выдавала логистическая регрессия. Поэтому в пайплайне указываем `LogisticRegression`:

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [20]:
# Пайплайн Word2Vec
pipeline_w2v = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline_w2v.fit(X_train_combined_w2v, y_train)
y_pred_w2v = pipeline_w2v.predict(X_test_combined_w2v)

In [21]:
# Пайплайн Word2Vec + sg
pipeline_w2v_sg = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline_w2v_sg.fit(X_train_combined_w2v_sg, y_train)
y_pred_w2v_sg = pipeline_w2v_sg.predict(X_test_combined_w2v_sg)

In [30]:
# Пайплайн FastText
pipeline_fasttext = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline_fasttext.fit(X_train_combined_fasttext, y_train)
y_pred_fasttext = pipeline_fasttext.predict(X_test_combined_fasttext)

In [35]:
# Пайплайн FastText + sg
pipeline_fasttext_sg = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline_fasttext_sg.fit(X_train_combined_fasttext_sg, y_train)
y_pred_fasttext_sg = pipeline_fasttext_sg.predict(X_test_combined_fasttext_sg)

In [42]:
# Пайплайн GloVe
pipeline_glove = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline_glove.fit(X_train_combined_glove, y_train)
y_pred_glove = pipeline_glove.predict(X_test_combined_glove)

#### Замер метрик

In [22]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, classification_report, confusion_matrix

In [24]:
print("Метрики для Word2Vec:\n")

# Accuracy, ROC-AUC
accuracy = accuracy_score(y_test, y_pred_w2v)
roc_auc = roc_auc_score(y_test, pipeline_w2v.predict_proba(X_test_combined_w2v), multi_class='ovr')
print("Accuracy:", accuracy)
print("ROC-AUC:", roc_auc)
print()

# Precision, Recall, F1
precision = precision_score(y_test, y_pred_w2v, average='micro')
recall = recall_score(y_test, y_pred_w2v, average='micro')
f1 = f1_score(y_test, y_pred_w2v, average='micro')

print("Micro metrics:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred_w2v)
print("Confusion Matrix:\n", conf_matrix)

Метрики для Word2Vec:

Accuracy: 0.8070175438596491
ROC-AUC: 0.9682197110784171

Micro metrics:
Precision: 0.8070175438596491
Recall: 0.8070175438596491
F1 Score: 0.8070175438596491
Confusion Matrix:
 [[14  2  1  1  0  0  3  0  0  0  1  0  0]
 [ 1 40  0  3  1  0  2  0  0  0  0  0  2]
 [ 0  1 31  1  0  0  0  2  0  2  1  1  1]
 [ 0  0  1 35  0  0  1  0  0  0  4  3  0]
 [ 0  1  1  0  8  0  0  0  0  1  0  0  1]
 [ 0  0  1  0  0 53  0  0  0  1  0  1  0]
 [ 0  1  3  2  0  1 56  0  0  1  0  0  2]
 [ 0  0  1  1  0  1  0  3  0  1  0  2  0]
 [ 0  0  0  0  0  0  1  0 51  0  0  0  1]
 [ 1  0  0  1  1  1  2  0  0 26  0  2  0]
 [ 1  1  1  2  0  0  0  0  0  0 29  3  0]
 [ 0  1  5  2  1  0  1  0  1  2  0 52  0]
 [ 1  0  1  5  0  0  2  0  0  0  0  1 16]]


In [25]:
print("Метрики для Word2Vec + Skip Gram:\n")

# Accuracy, ROC-AUC
accuracy = accuracy_score(y_test, y_pred_w2v_sg)
roc_auc = roc_auc_score(y_test, pipeline_w2v_sg.predict_proba(X_test_combined_w2v_sg), multi_class='ovr')
print("Accuracy:", accuracy)
print("ROC-AUC:", roc_auc)
print()

# Precision, Recall, F1
precision = precision_score(y_test, y_pred_w2v_sg, average='micro')
recall = recall_score(y_test, y_pred_w2v_sg, average='micro')
f1 = f1_score(y_test, y_pred_w2v_sg, average='micro')

print("Micro metrics:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred_w2v_sg)
print("Confusion Matrix:\n", conf_matrix)

Метрики для Word2Vec + Skip Gram:

Accuracy: 0.8674463937621832
ROC-AUC: 0.9803196275721319

Micro metrics:
Precision: 0.8674463937621832
Recall: 0.8674463937621832
F1 Score: 0.8674463937621832
Confusion Matrix:
 [[16  1  0  0  1  0  2  0  0  1  0  1  0]
 [ 1 42  0  0  1  0  1  0  0  1  0  0  3]
 [ 0  0 36  1  0  0  0  0  0  1  1  0  1]
 [ 0  0  0 39  0  0  1  0  0  0  3  1  0]
 [ 0  0  0  1 10  0  0  0  0  1  0  0  0]
 [ 0  0  0  1  0 53  0  0  0  1  0  1  0]
 [ 0  1  0  0  0  0 63  0  0  1  0  0  1]
 [ 0  0  1  2  0  0  1  1  0  0  0  4  0]
 [ 0  0  0  0  0  0  1  0 51  0  0  0  1]
 [ 0  0  0  0  1  0  2  2  0 28  0  1  0]
 [ 0  1  0  1  0  0  0  0  0  0 34  1  0]
 [ 0  0  1  2  0  0  0  0  2  2  0 57  1]
 [ 1  2  0  3  0  0  2  1  0  2  0  0 15]]


In [31]:
print("Метрики для FastText:\n")

# Accuracy, ROC-AUC
accuracy = accuracy_score(y_test, y_pred_fasttext)
roc_auc = roc_auc_score(y_test, pipeline_fasttext.predict_proba(X_test_combined_fasttext), multi_class='ovr')
print("Accuracy:", accuracy)
print("ROC-AUC:", roc_auc)
print()

# Precision, Recall, F1
precision = precision_score(y_test, y_pred_fasttext, average='micro')
recall = recall_score(y_test, y_pred_fasttext, average='micro')
f1 = f1_score(y_test, y_pred_fasttext, average='micro')

print("Micro metrics:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred_fasttext)
print("Confusion Matrix:\n", conf_matrix)

Метрики для FastText:

Accuracy: 0.8362573099415205
ROC-AUC: 0.9774748488976457

Micro metrics:
Precision: 0.8362573099415205
Recall: 0.8362573099415205
F1 Score: 0.8362573099415205
Confusion Matrix:
 [[16  1  1  1  1  0  1  1  0  0  0  0  0]
 [ 0 37  0  2  0  0  7  0  0  1  0  0  2]
 [ 0  1 32  1  0  0  0  0  0  4  1  0  1]
 [ 2  0  2 35  0  0  2  0  0  0  2  0  1]
 [ 1  0  1  0  8  0  0  0  0  1  0  0  1]
 [ 0  0  0  0  0 53  0  0  0  0  0  1  2]
 [ 0  2  1  0  0  0 62  0  0  1  0  0  0]
 [ 0  0  1  0  0  0  1  4  0  0  0  3  0]
 [ 0  1  0  0  0  0  3  0 49  0  0  0  0]
 [ 3  1  0  0  1  1  2  3  0 21  0  2  0]
 [ 0  1  0  0  0  0  0  0  0  0 36  0  0]
 [ 0  0  0  3  1  0  1  0  0  1  0 58  1]
 [ 1  0  1  2  0  0  2  1  0  1  0  0 18]]


In [36]:
print("Метрики для FastText + Skip Gram:\n")

# Accuracy, ROC-AUC
accuracy = accuracy_score(y_test, y_pred_fasttext_sg)
roc_auc = roc_auc_score(y_test, pipeline_fasttext_sg.predict_proba(X_test_combined_fasttext_sg), multi_class='ovr')
print("Accuracy:", accuracy)
print("ROC-AUC:", roc_auc)
print()

# Precision, Recall, F1
precision = precision_score(y_test, y_pred_fasttext_sg, average='micro')
recall = recall_score(y_test, y_pred_fasttext_sg, average='micro')
f1 = f1_score(y_test, y_pred_fasttext_sg, average='micro')

print("Micro metrics:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred_fasttext_sg)
print("Confusion Matrix:\n", conf_matrix)

Метрики для FastText + Skip Gram:

Accuracy: 0.8947368421052632
ROC-AUC: 0.9839140873949538

Micro metrics:
Precision: 0.8947368421052632
Recall: 0.8947368421052632
F1 Score: 0.8947368421052632
Confusion Matrix:
 [[19  0  0  1  0  0  2  0  0  0  0  0  0]
 [ 1 43  0  2  0  0  0  0  0  0  0  0  3]
 [ 0  1 37  0  0  0  0  0  0  1  1  0  0]
 [ 1  1  1 39  0  0  0  0  0  0  0  1  1]
 [ 0  0  0  0 12  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 53  0  0  0  0  1  2  0]
 [ 0  1  0  0  0  0 63  0  0  1  0  0  1]
 [ 0  0  0  0  0  0  2  2  0  2  0  3  0]
 [ 1  0  0  0  0  0  0  0 51  0  0  1  0]
 [ 1  0  0  1  0  1  1  1  0 29  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0 35  1  0]
 [ 0  1  0  2  1  0  0  0  1  0  1 58  1]
 [ 1  1  0  2  0  0  3  0  0  0  1  0 18]]


In [43]:
print("Метрики для GloVe:\n")

# Accuracy, ROC-AUC
accuracy = accuracy_score(y_test, y_pred_glove)
roc_auc = roc_auc_score(y_test, pipeline_glove.predict_proba(X_test_combined_glove), multi_class='ovr')
print("Accuracy:", accuracy)
print("ROC-AUC:", roc_auc)
print()

# Precision, Recall, F1
precision = precision_score(y_test, y_pred_glove, average='micro')
recall = recall_score(y_test, y_pred_glove, average='micro')
f1 = f1_score(y_test, y_pred_glove, average='micro')

print("Micro metrics:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred_glove)
print("Confusion Matrix:\n", conf_matrix)

Метрики для GloVe:

Accuracy: 0.2124756335282651
ROC-AUC: 0.6613072657129375

Micro metrics:
Precision: 0.2124756335282651
Recall: 0.2124756335282651
F1 Score: 0.2124756335282651
Confusion Matrix:
 [[ 0  3  0  0  0  6 10  0  1  0  0  2  0]
 [ 0 24  0  0  0  4 18  0  0  0  1  2  0]
 [ 0  6  0  1  1  4 15  0  3  0  7  3  0]
 [ 0  5  0  3  0 11 14  0  0  1  2  5  3]
 [ 0  7  0  0  1  0  3  0  0  1  0  0  0]
 [ 0  3  0  0  0  6 37  0  0  0  6  4  0]
 [ 0  2  0  0  0  6 53  0  1  0  1  3  0]
 [ 0  2  0  0  1  3  1  0  0  0  2  0  0]
 [ 0  2  0  2  1  4 37  0  1  0  2  4  0]
 [ 0  3  0  1  0  2 24  0  1  0  1  2  0]
 [ 0  1  0  3  0  8  8  0  0  0 15  2  0]
 [ 0  5  0  4  1 12 29  0  3  1  4  6  0]
 [ 0  5  0  2  2  2 11  0  0  0  2  2  0]]


По итогу на комбинированном наборе фичей (эвристики + текстовые) логистическая регрессия показала лучшие метрики для $\text{FastText + Skip Gram}$, однако время обучения модели оказалось достаточно большим.<br>
Оптимальным подходом по соотношению времени обучения и значений метрик можно назвать $\text{Word2Vec + Skip Gram}$.<br>
Предобученный $\text{GloVe}$ не подошел для нашей задачи предсказания авторства текстов из классической отечественной литературы.

Сохранение лучших моделей в `.pkl` файлы:

In [ ]:
!pip install joblib

In [ ]:
from joblib import dump, load

dump(model_w2v_sg, 'model_w2v_sg.pkl')
dump(pipeline_w2v_sg, 'pipeline_w2v_sg.pkl')